In [1]:
import operator
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report

In [2]:
s = open("C:/Users/1/Desktop/alternative.txt", "r", encoding="utf-8")
lines = s.readlines()

In [3]:
#no suffixes
arr = []
for i in range(len(lines)):
    line = lines[i].strip("\n")
    items = line.split("/")
    p1 = items[1]
    pos1 = items[2]
    p2 = items[4]
    pos2 = items[5]
    suf = items[7]
    t = items[-1]
    tl = (p1,pos1,p2,pos2,suf,t)
    arr.append(tl)

In [4]:
dict1 = {z:arr.count(z) for z in arr}

In [5]:
dict2 = {}
for k,v in dict1.items():
    if (k[0], k[1], k[2], k[3], k[4]) not in dict2.keys():
        dict2[(k[0], k[1], k[2], k[3], k[4])] = [(k[5], v)]
    else:
        dict2[(k[0], k[1], k[2], k[3], k[4])].append((k[5], v))

In [6]:
dict3 = {}
for k,v in dict2.items():
    all_count = 0
    for x in v:
        all_count += x[1]
    dict3[k] = {'N': 0, 'NN' : 0, 'AN': 0}
    for x in v:
        dict3[k][x[0]] = x[1] / all_count

In [7]:
#distance between compound parts: not normalized
def dist_not_norm(a,b):
    string1 = a[::-1]
    string2 = b[::-1]
    distance = 0
    n1 = len(string1)
    n2 = len(string2)
    if n1 >= n2:
        for i in range(n1):
            if i < n2:
                if string1[i] == string2[i]:
                    distance += 1
                else:
                    return distance
        if distance == n1 and distance == n2:
            return 0
        else:
            return distance
    else:
        for i in range(n2):
            if i < n1:
                if string2[i] == string1[i]:
                    distance += 1
                else:
                    return distance
        return distance

In [8]:
#distance between compound parts: normalized
def dist_norm(a,b):
    return 1-(dist_not_norm(a,b)/((len(a)+len(b))/2))

In [9]:
#general distance between vectors
def all_dist(vector_1,vector_2):
    cur_dist = 0
    for i in range((len(vector_1))):
        if i == 0 or i == 3:
            ith_dist = dist_norm(vector_1[i], vector_2[i])
            cur_dist += ith_dist * ith_dist
        else:
            if vector_1[i] != vector_2[i]:
                cur_dist += 1
    return cur_dist

In [10]:
matrixX = []
matrixY = []
for k,v in dict3.items():
    matrixX.append(k)
    matrixY.append([v['AN'], v['N'], v['NN']])

In [11]:
matrixX[:3]

[('Engel', '<NN>', 'Bild', '<+NN>', 's'),
 ('Wolke', '<NN>', 'Schatten', '<+NN>', 'n'),
 ('Sonne', '<NN>', 'Strahl', '<+NN>', 'n')]

In [12]:
matrixY[:3]

[[0, 1.0, 0], [0, 0.5, 0.5], [0, 0, 1.0]]

In [13]:
X = np.array(matrixX)
y = np.array(matrixY)

In [14]:
kf = KFold(n_splits=3, shuffle=True)

In [15]:
kf.get_n_splits(X)

3

In [16]:
all_predicts = []
tests = []

for train_index, test_index in kf.split(X[:20]):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    predictions = []
    for vector in X_test:
        distances = []
        types = []
        for x in range(len(X_train)):
            distance = all_dist(X_train[x], vector)
            d_t = (distance, list(y_train[x]))
            distances.append(d_t)
            types.append(list(y_train[x]))
        distances_sort = sorted(distances)
        types_sort = np.array([0., 0., 0.])
        for item in distances_sort[:3]:
            types_sort += np.array(item[1])
        mean = types_sort / 3
        print (mean)

[ 0.16666667  0.16666667  0.66666667]
[ 0.16666667  0.5         0.33333333]
[ 0.   0.5  0.5]
[ 0.16666667  0.16666667  0.66666667]
[ 0.16666667  0.83333333  0.        ]
[ 0.   0.5  0.5]
[ 0.16666667  0.16666667  0.66666667]
[ 0.22727273  0.43939394  0.33333333]
[ 0.11111111  0.88888889  0.        ]
[ 0.22727273  0.77272727  0.        ]
[ 0.5  0.5  0. ]
[ 0.11111111  0.88888889  0.        ]
[ 0.11111111  0.88888889  0.        ]
[ 0.11111111  0.88888889  0.        ]
[ 0.33333333  0.38888889  0.27777778]
[ 0.39393939  0.60606061  0.        ]
[ 0.25  0.25  0.5 ]
[ 0.58333333  0.41666667  0.        ]
[ 0.   0.5  0.5]
[ 0.33333333  0.16666667  0.5       ]
